# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [2]:
%pip install datasets causal_conv1d

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.2 MB/s eta 0:00

# Lab

In [38]:
import VAE, importlib
importlib.reload(VAE)
from VAE import MambaVAE

In [22]:
import torch, InitMamba, importlib
importlib.reload(InitMamba)
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [32]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["\nMy name is Peter. I'm happy."], return_tensors='pt', max_length = 9, padding = 'max_length').to('cuda')

In [39]:
vae = MambaVAE().cuda()
vae.load_state_dict(torch.load('./results/checkpoint-18288/model.pth', weights_only=True))

<All keys matched successfully>

In [40]:
vae.train()
res = vae(**inputs)
tokenizer.batch_decode(res[2].argmax(-1))

["My name is Peter. I'm happy. I"]

In [20]:
inputs['input_ids'].shape

torch.Size([1, 128])

In [31]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model2 = InitMamba.MambaModel.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()
# model2.train()

# hidden_states = model2(**inputs, layer_range = range(23))[0]
# res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'], layer_range = range(23, 24), output_ssm_last_states = True)
res1 = model1(**inputs, labels = inputs['input_ids'], output_ssm_last_states = True)
# ssm_last_states = res1.ssm_last_states
# print(res1[0])
print((res1.ssm_last_states).pow(2).sum()/2)

tensor(4849.4121, device='cuda:0', grad_fn=<DivBackward0>)


In [42]:
res1.ssm_last_states.abs().mean()

tensor(0.1053, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
hidden_states = model2(**inputs, layer_range = range(1), inputs_ssm_states = ssm_last_states)[0]
res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'],
              layer_range = range(1, 24),
              output_ssm_last_states = True,
              inputs_ssm_states = ssm_last_states)
res = model1(**inputs,
             output_ssm_last_states = True,
             inputs_ssm_states = ssm_last_states)
print(res.logits-res1.logits)
print(res.logits)

In [26]:
import torch
print((0.15*torch.randn(32, 768)).pow(2).sum()/2)

tensor(274.9822)


In [ ]:
(0.01*torch.randn(32, 768)).abs().mean()

In [3]:
!python Trainer.py

{'----> logits_loss': 0.621582567691803, 'kl_loss': 2126.28271484375, 'kl_decay': 1e-08, 'epoch': 1.9}
{'loss': 0.6216, 'grad_norm': 0.8412707448005676, 'learning_rate': 5.325232308791995e-06, 'epoch': 1.9}
{'----> logits_loss': 0.6062508225440979, 'kl_loss': 2196.130126953125, 'kl_decay': 1e-08, 'epoch': 1.92}
{'loss': 0.6063, 'grad_norm': 0.9093316793441772, 'learning_rate': 4.4317369549678345e-06, 'epoch': 1.92}
{'----> logits_loss': 0.59754878282547, 'kl_loss': 2209.45703125, 'kl_decay': 1e-08, 'epoch': 1.94}
{'loss': 0.5976, 'grad_norm': 0.9384087324142456, 'learning_rate': 3.538241601143674e-06, 'epoch': 1.94}
{'----> logits_loss': 0.6105940937995911, 'kl_loss': 2214.2197265625, 'kl_decay': 1e-08, 'epoch': 1.95}
{'loss': 0.6106, 'grad_norm': 0.8424633145332336, 'learning_rate': 2.6447462473195144e-06, 'epoch': 1.95}
{'----> logits_loss': 0.6135134100914001, 'kl_loss': 2207.02783203125, 'kl_decay': 1e-08, 'epoch': 1.97}
{'loss': 0.6135, 'grad_norm': 0.8150379657745361, 'learning_r